Wprowadzenie do Exact Cover Problemu

Stworzenie instancji problemu Exact Cover:

In [3]:
A = {1,4,7}
B = {1,4}
C = {4,5,7}
D = {3,5,6}
E = {2,3,6,7}
F = {2,7}
routes = [A,B,C,D,E,F]

In [4]:
from utils import EXACTCOVER
exact_cover = EXACTCOVER(routes)


ImportError: cannot import name 'ExactCover' from 'utils' (/home/jrozycki/Documents/Praca/jupyternotebook/utils.py)

In [ ]:
exact_cover.find_best_parameters()

In [1]:
from utils import EXACTCOVER
import json
with open('conf.json') as f:
    config = json.load(f)

Algo = EXACTCOVER(routes, config)
Algo.find_best_parameters()

  0%|          | 0/300 [00:00<?, ?it/s]



graph_data: [] 

Najlepsze rozwiazanie to: 101010


0